In [1]:
#Creating a dataframe from website data
Crop_df <- read.csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-RP0203EN-SkillsNetwork/labs/Final%20Project/Annual_Crop_Data.csv', colClasses=c(YEAR="character"))
    Bank_df <- read.csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-RP0203EN-SkillsNetwork/labs/Final%20Project/Daily_FX.csv', colClasses=c(date="character"))

    head(Crop_df)
    head(Bank_df)

Warning message in read.table(file = file, header = header, sep = sep, quote = quote, :
“not all columns named in 'colClasses' exist”

,CD_ID,YEAR,CROP_TYPE,GEO,SEEDED_AREA,HARVESTED_AREA,PRODUCTION,AVG_YIELD
,<int>,<chr>,<fct>,<fct>,<int>,<int>,<int>,<int>
1,0,1965-12-31,Barley,Alberta,1372000,1372000,2504000,1825
2,1,1965-12-31,Barley,Canada,2476800,2476800,4752900,1920
3,2,1965-12-31,Barley,Saskatchewan,708000,708000,1415000,2000
4,3,1965-12-31,Canola,Alberta,297400,297400,215500,725
5,4,1965-12-31,Canola,Canada,580700,580700,512600,885
6,5,1965-12-31,Canola,Saskatchewan,224600,224600,242700,1080


,DFX_ID,DATE,FXUSDCAD
,<int>,<fct>,<dbl>
1,0,2017-01-03,1.3435
2,1,2017-01-04,1.3315
3,2,2017-01-05,1.3244
4,3,2017-01-06,1.3214
5,4,2017-01-09,1.3240
6,5,2017-01-10,1.3213


In [2]:
#Creating database connection
dsn_driver <- "IBM DB2 ODBC Driver"
dsn_database <- "bludb"           
dsn_hostname <- "98538591-7217-4024-b027-8baa776ffad1.c3n41cmd0nqnrk39u98g.databases.appdomain.cloud" 
dsn_port <- "30875"    
dsn_protocol <- "TCPIP"            
dsn_uid <- "vfx98987"        
dsn_pwd <- "Password"      
dsn_security <- "ssl"



In [3]:
library(RODBC);
#Connecting to the databse
conn_path <- paste("DRIVER=",dsn_driver,
                  ";DATABASE=",dsn_database,
                  ";HOSTNAME=",dsn_hostname,
                  ";PORT=",dsn_port,
                  ";PROTOCOL=",dsn_protocol,
                  ";UID=",dsn_uid,
                  ";PWD=",dsn_pwd,
                  ";SECURITY=",dsn_security,        
                    sep="")
conn <- odbcDriverConnect(conn_path)
conn

RODBC Connection 1
Details:
  case=nochange
  DRIVER=IBM DB2 ODBC DRIVER
  UID=vfx98987
  PWD=******
  DATABASE=bludb
  HOSTNAME=98538591-7217-4024-b027-8baa776ffad1.c3n41cmd0nqnrk39u98g.databases.appdomain.cloud
  PORT=30875
  PROTOCOL=TCPIP
  SECURITY=SSL

In [4]:
#Viewing database driver information
sql.info <- sqlTypeInfo(conn)
conn.info <- odbcGetInfo(conn)
conn.info["DBMS_Name"]
conn.info["DBMS_Ver"]
conn.info["Driver_ODBC_Ver"]

DBMS_Name 
"DB2/LINUXX8664"

DBMS_Ver 
"11.05.0700"

Driver_ODBC_Ver 
        "03.51"

In [5]:
#checking if the tables exist and dropping them if they exist
myschema <- "VFX98987" 
    tables <- c("CROP", "BANKEX") 

    for (table in tables) {
        # Drop tables if they already exist
        out <- sqlTables(conn, tableType = "TABLE",
                          tableName = table)
        if (nrow(out)>0) {
            err <- sqlDrop(conn, table,
                            errors=FALSE)  
            if (err==-1) {
                cat("An error has occurred.\n")
                err.msg <- odbcGetErrMsg(conn)
                for (error in err.msg) { 
                    cat(error,"\n")
                }
            } 
            else {
                cat ("Table: ",table," was dropped\n")
            }
        }
        else {
              cat ("Table: ", table," does not exist\n")
        }
    }



Table:  CROP  was dropped
Table:  BANKEX  was dropped


In [6]:
#Creat table CROP
df1 <- sqlQuery(conn, "CREATE TABLE CROP (
                            CD_ID VARCHAR(6) NOT NULL, 
                            YEAR VARCHAR(10) NOT NULL, 
                            CROP_TYPE VARCHAR(50) NOT NULL, 
                            GEO VARCHAR(50) NOT NULL, 
                            SEED_AREA INTEGER NOT NULL,
                            HARVESTED_AREA INTEGER NOT NULL,
                            PRODUCTION INTEGER NOT NULL,
                            AVG_YEILD INTEGER NOT NULL,
                            PRIMARY KEY (CD_ID))", 
                errors=FALSE)

In [7]:
if (df1 == -1){
  cat ("An error has occurred.\n")
  msg <- odbcGetErrMsg(conn)
  print (msg)
} else {
  cat ("Table was created successfully.\n")
}

Table was created successfully.


In [8]:
#Creat table BANKEX
df2 <- sqlQuery(conn, "CREATE TABLE BANKEX (
                            DFX_ID CHAR(6) NOT NULL, 
                            DATE VARCHAR(10) NOT NULL, 
                            FXUSDCAD FLOAT(6) , 
                            
                            PRIMARY KEY (DFX_ID))", 
                errors=FALSE)

In [9]:
if (df2 == -1){
  cat ("An error has occurred.\n")
  msg <- odbcGetErrMsg(conn)
  print (msg)
} else {
  cat ("Table was created successfully.\n")
}

Table was created successfully.


In [10]:
#Saving crop data into created table crop
sqlSave(conn, Crop_df, "CROP" ,append=TRUE, fast=FALSE,rownames=FALSE, colnames=FALSE, verbose=FALSE)

In [11]:
#Saving bank data into created table bankex
sqlSave(conn, Bank_df, "BANKEX" ,append=TRUE, fast=FALSE,rownames=FALSE, colnames=FALSE, verbose=FALSE)

In [12]:
#Checking data size or number of rows in given data
query = "SELECT COUNT(CD_ID) FROM CROP"
    sqlQuery(conn,query)
    
query = "SELECT COUNT(DFX_ID) FROM BANKEX"
    sqlQuery(conn,query)

,1
,<int>
1,672


,1
,<int>
1,1033


In [13]:
#Using SQL to display initial data
query <- "SELECT * FROM CROP LIMIT 6;"
    view <- sqlQuery(conn,query)
    view

,CD_ID,YEAR,CROP_TYPE,GEO,SEED_AREA,HARVESTED_AREA,PRODUCTION,AVG_YEILD
,<int>,<fct>,<fct>,<fct>,<int>,<int>,<int>,<int>
1,0,1965-12-31,Barley,Alberta,1372000,1372000,2504000,1825
2,1,1965-12-31,Barley,Canada,2476800,2476800,4752900,1920
3,2,1965-12-31,Barley,Saskatchewan,708000,708000,1415000,2000
4,3,1965-12-31,Canola,Alberta,297400,297400,215500,725
5,4,1965-12-31,Canola,Canada,580700,580700,512600,885
6,5,1965-12-31,Canola,Saskatchewan,224600,224600,242700,1080


In [14]:
#Checking the various types of crops in the table using SQL and R
query <- "SELECT DISTINCT(CROP_TYPE) FROM CROP;"
    view <- sqlQuery(conn,query)
    view

,CROP_TYPE
,<fct>
1,Barley
2,Canola
3,Rye
4,Wheat


In [15]:
#Checking all details where crop is Rye for 6 entries
query <- "SELECT * FROM CROP WHERE CROP_TYPE='Rye' LIMIT 6;"
    view <- sqlQuery(conn,query)
    view

,CD_ID,YEAR,CROP_TYPE,GEO,SEED_AREA,HARVESTED_AREA,PRODUCTION,AVG_YEILD
,<int>,<fct>,<fct>,<fct>,<int>,<int>,<int>,<int>
1,6,1965-12-31,Rye,Alberta,81000,81000,116400,1435
2,7,1965-12-31,Rye,Canada,323900,323900,453400,1400
3,8,1965-12-31,Rye,Saskatchewan,166000,166000,224000,1350
4,18,1966-12-31,Rye,Alberta,70000,70000,109000,1555
5,19,1966-12-31,Rye,Canada,293400,293400,437600,1490
6,20,1966-12-31,Rye,Saskatchewan,161000,161000,228600,1420


In [16]:
#Checking which crops have an average yeild greater than or equal to 3000 hectare
query <- "SELECT DISTINCT(CROP_TYPE) FROM CROP WHERE AVG_YEILD>=3000;"
    view <- sqlQuery(conn,query)
    view


,CROP_TYPE
,<fct>
1,Barley
2,Rye
3,Wheat


In [17]:
#Checking the first and the last dates of each year

query <- "SELECT MIN(YEAR) FIRST, MAX(YEAR) LAST FROM CROP;"
    view <- sqlQuery(conn,query)
    view


query <- "SELECT MIN(DATE) FIRST, MAX(DATE) LAST FROM BANKEX;"
    view <- sqlQuery(conn,query)
    view

,FIRST,LAST
,<fct>,<fct>
1,1965-12-31,2020-12-31


,FIRST,LAST
,<fct>,<fct>
1,2017-01-03,2021-02-18


In [18]:
#Selecting Top 10 years of Wheat production in Saskatechewan 
query <- "SELECT YEAR, HARVESTED_AREA FROM CROP WHERE GEO='Saskatchewan' AND CROP_TYPE='Wheat' ORDER BY HARVESTED_AREA DESC LIMIT 10;"
    view <- sqlQuery(conn,query)
    view


,YEAR,HARVESTED_AREA
,<fct>,<int>
1,1986-12-31,8765400
2,1991-12-31,8595120
3,1983-12-31,8377000
4,1985-12-31,8357000
5,1990-12-31,8308200
6,1992-12-31,8284100
7,1987-12-31,8235400
8,1989-12-31,8097700
9,1984-12-31,8094000


In [19]:
#Counting number of years when Barley yeild was greater than or equal to 2000KG in Canada
query <- "select count(distinct(Year)) from CROP where GEO='Canada' and CROP_TYPE='Barley' and AVG_YEILD>=2000;"
view <- sqlQuery(conn,query)
view

,1
,<int>
1,52


In [20]:
#Selecting land seeded but not harvested with barley in Alberta since year 2000
query <- "select YEAR(YEAR),GEO,CROP_TYPE, SEED_AREA, HARVESTED_AREA,100*(SEED_AREA-HARVESTED_AREA)/SEED_AREA as UNHARVESTED_AREA from CROP where year(year)>=2000 and GEO='Alberta' and CROP_TYPE='Barley';"
view <- sqlQuery(conn,query)
view

,1,GEO,CROP_TYPE,SEED_AREA,HARVESTED_AREA,UNHARVESTED_AREA
,<int>,<fct>,<fct>,<int>,<int>,<int>
1,2000,Alberta,Barley,2185300,1740100,20
2,2001,Alberta,Barley,1983000,1618700,18
3,2002,Alberta,Barley,2169100,1127000,48
4,2003,Alberta,Barley,2144800,1788700,16
5,2004,Alberta,Barley,1910100,1598500,16
6,2005,Alberta,Barley,1709800,1456900,14
7,2006,Alberta,Barley,1657100,1375900,16
8,2007,Alberta,Barley,1962700,1728000,11
9,2008,Alberta,Barley,1679400,1517600,9


In [21]:
#Selecting average value of Canadian Dollar over the last three years
query <- "SELECT  AVG(FXUSDCAD) AS FX_DAILY_AVG_CAD 
    FROM  BANKEX limit 3;
    "
    view <- sqlQuery(conn,query)
    view



,FX_DAILY_AVG_CAD
,<dbl>
1,1.314358


In [22]:
#Closing the Connection
close(conn)